In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29,1,0.0
1,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2023-01-30,1,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2023-01-31,1,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-01,1,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81085,5.0,10.0,7.0,10.0,13.0,3.0,3.0,2.0,1.0,2.0,...,2.0,2.0,2.0,2.0,6.0,0.0,1.0,2023-11-26,265,4.0
81086,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,...,13.0,5.0,2.0,3.0,5.0,4.0,1.0,2023-11-27,265,4.0
81087,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,6.0,4.0,2.0,6.0,2.0,8.0,2.0,2023-11-28,265,4.0
81088,1.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,1.0,2.0,...,14.0,3.0,4.0,8.0,5.0,2.0,4.0,2023-11-29,265,1.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 5, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(24380, 674)
y_train.shape=(24380,)
X_test.shape=(56710, 674)
y_test.shape=(56710,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-02-15 19:32:03,871] A new study created in memory with name: no-name-1962de8e-7ba1-4aea-add4-28b1a42b3745
[I 2024-02-15 19:32:31,718] Trial 0 finished with value: 4.55646811268468 and parameters: {'num_leaves': 120, 'feature_fraction': 0.9412027070352647, 'bagging_fraction': 0.8774006544851944, 'min_child_samples': 56}. Best is trial 0 with value: 4.55646811268468.
[I 2024-02-15 19:33:09,252] Trial 1 finished with value: 4.116512725439068 and parameters: {'num_leaves': 174, 'feature_fraction': 0.7525518821746748, 'bagging_fraction': 0.34627254102130567, 'min_child_samples': 5}. Best is trial 1 with value: 4.116512725439068.
[I 2024-02-15 19:33:35,165] Trial 2 finished with value: 4.469455602928873 and parameters: {'num_leaves': 154, 'feature_fraction': 0.5863388597883744, 'bagging_fraction': 0.3202298679745207, 'min_child_samples': 62}. Best is trial 1 with value: 4.116512725439068.
[I 2024-02-15 19:33:58,876] Trial 3 finished with value: 4.630588728434652 and parameters: {'num

In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 71, 'feature_fraction': 0.5776715420897691, 'bagging_fraction': 0.9742370874605031, 'min_child_samples': 34}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.5776715420897691, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5776715420897691
[LightGBM] [Warning] bagging_fraction is set=0.9742370874605031, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9742370874605031
[LightGBM] [Warning] feature_fraction is set=0.5776715420897691, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5776715420897691
[LightGBM] [Warning] bagging_fraction is set=0.9742370874605031, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9742370874605031
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.115668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 152006
[LightGBM] [Info] Number of data points in the train set: 24380, number of used features: 675
[LightGBM] [Info] Start training from score 11.233839


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001F066AC09A0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.9742370874605031,
                               feature_fraction=0.5776715420897691,
                               min_child_samples=34, num_leaves=71))])

In [8]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.5776715420897691, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5776715420897691
[LightGBM] [Warning] bagging_fraction is set=0.9742370874605031, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9742370874605031
test_mae=2.8214


In [9]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [10]:
plot_one_sample(
    example_id=3979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)